In [ ]:
"""
Author: Liam Bogucki
Email: lboguck@uwo.ca
First Written: Tuesday, June 7, 2024
Last Modified: Sunday, September 14, 2025
Program Purpose: To process the MODIS data into a useable 0.5 degree dataset.
"""

In [ ]:
# Importing appropriate libraries.
import numpy as np
import matplotlib.pyplot as plt
from pyhdf.SD import SD, SDC
import scipy.stats as stats

In [ ]:
#The purpose of this block is to process the hdf file into a numpy array

#Opening the MODIS file to read
hdf = SD("MCD12C1.hdf", SDC.READ)

#Reading the datasets in from the opened hdf
dict_datasets = hdf.datasets()

#Listing the datasets to investigate the parts
i = 0
for key in dict_datasets:
    info = hdf.select(key)
    print(f"({i}){key} : attributes:{info.attributes()}")
    i+=1


# Selecting the 6th dataset gets us the same data as used by Ahlstrom (Modis type 3 majority land cover type).
dataset = list(dict_datasets.keys())[6]
modis_dataset = hdf.select(dataset)
modis_data = modis_dataset.get()

#Investigating the dimensions of the data to decide how to resize it (conducted below)
print(np.shape(modis_data))

In [ ]:
#This block is concerend with getting the desired 0.5 degree resolution of the data from the existing (3600, 7200)

#Reshaping the array down to the correct size to chunk it (10x10)
modis_data_reshaped = modis_data.reshape(360, 10, 720, 10)

#Transposing to place 360, 720 in front
modis_data_transposed = modis_data_reshaped.transpose(0, 2, 1, 3)

#Reshaping to correct shape to get mode data from chunks
modis_data_reshaped = modis_data_transposed.reshape(360, 720, -1)

#Getting the mode value for the grid cells
modis_data_mode = stats.mode(modis_data_reshaped, axis=-1)[0]

#Verifying the map and the values
print(np.shape(modis_data_mode)) #The shape
print(np.unique(modis_data_mode)) #The unique values
plt.imshow(modis_data_mode) #Plot to verify the look
plt.colorbar()
plt.show()

#Saving as npy array for later use
np.save("MODIS_0.5Deg_Type3_Mode.npy", modis_data_mode)

